# LELA70331 Computational Linguistics Week 11

This week we are going to take a look at Syntactic analysis, starting with part-of-speech tagging


### Tagged corpora¶
In looking to understand part of speech tagging, it is useful to start by looking at some human (rather than machine) tagged data. NLTK contains a number of corpora. We can import one of these as follows:

In [ ]:
import nltk
nltk.download('brown')
from nltk.corpus import brown

In [ ]:
brown.tagged_words()[1:25]

## Inspecting tagged corpora

Inspecting human tagged corpora can be useful for both linguistic research and for building taggers. We can use the NLTK toolkit to do this. 

Most straightforwardly we can look at the frequency with which particular words are given a tag (we will return to this later when we come to build a tagger).

In [ ]:
sent = [("the","DET"),("man","NOUN"),("walked","VERB"),("the","DET"),("dog","NOUN")]

In [ ]:
cfd1 = nltk.ConditionalFreqDist(sent)
cfd1['the']

When we apply this to whole corpora, it becomes useful.

In [ ]:
brown_tagged = brown.tagged_words(tagset='universal')
cfd1 = nltk.ConditionalFreqDist(brown_tagged)
cfd1['the']

We can extend this to look at the frequency with which particular word classes precede particular words

In [ ]:
brown_tagged = brown.tagged_words(tagset='universal')
tags = [b[1] for (a, b) in nltk.bigrams(brown_tagged) if a[0] == 'the']
fd = nltk.FreqDist(tags)
fd.tabulate()

## Building an automatic tagger

A very simple approach to automated tagging that actually works quite well is to find the most common tag for each word in a training corpus (as we did above) and just tag all occurences of each word with its most common tag:

In [ ]:
brown_tagged_sents = brown.tagged_sents(tagset='universal')

In [ ]:
unigram_tagger = nltk.UnigramTagger(brown_tagged_sents)

In [ ]:
brown_tagged = brown.tagged_words(tagset='universal')
tags = [b[1] for (a, b) in nltk.bigrams(brown_tagged) if a[0] == 'the']
fd = nltk.FreqDist(tags)
fd.tabulate()

We can formally evaluate this by splitting our data into a training set and a testing set. We obtain the by-word tag frequencies from the training set and evaluate by tagging the test set and comparing our predicted tags to the human tags.

In [ ]:
training_set_size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:training_set_size]
test_sents = brown_tagged_sents[training_set_size:]
unigram_tagger = nltk.UnigramTagger(train_sents)
unigram_tagger.evaluate(test_sents)

### Regular expression based tagging

As a next step we want to use a more intelligent way to deal with words we haven't seen before, but making use of their orthography and/or morphology. Write regular expressions to classify words in this way and see if you can improve performance. I've added one example rule to get you started.

In [ ]:
patterns = [
    (r'.*ing$', 'VERB'),
      ]

In [ ]:
t0 = nltk.DefaultTagger('NOUN')
t1 = nltk.RegexpTagger(patterns, backoff=t0)
t2 = nltk.UnigramTagger(train_sents, backoff=t1)
t2.evaluate(test_sents)

### Looking at the context

We want to improve this, and an obvious next step is to give the tag that is most frequent for this word when it follows the previous word. The problem is this doesn't do very well. Any idea why?

In [ ]:
bigram_tagger = nltk.BigramTagger(train_sents)
bigram_tagger.evaluate(test_sents)

We can still make use of the bigram information by combining it with the unigram tagger via a process known as backing off - for each word we check whether we have seen that word and preceding word in our training data. If we have then we tag it with the most frequent tag for that word in that context. If we haven't seen it then we tag the word with its most frequent tag regardless of context. And if we haven't seen the word before we tag it as a noun.

In [ ]:
t0 = nltk.DefaultTagger('NOUN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t2.evaluate(test_sents)

### NLTK's Averaged Perceptron tagger

NLTKs default prebuilt tagger uses a Perceptron just like that we have been using for other tasks on the module. For more information on this approach see here: https://explosion.ai/blog/part-of-speech-pos-tagger-in-python


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

It can be run straightforwardly like this:

In [ ]:
text = nltk.word_tokenize("And now for something completely different")
nltk.pos_tag(text, tagset="universal")

## Syntactic Parsing

We are going once again to use tools from NLTK, which we need to import as follows: 

In [ ]:
import nltk
from nltk.parse.generate import generate
from nltk import CFG, Tree
nltk.download('punkt')


We can define phrase structure grammars using rewrite rules (see week 10 lecture for a definition) as follows: 

In [ ]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Pronoun
    VP -> V NP 
    Det -> 'the' 
    Pronoun -> 'I'
    N -> 'dishes'  
    V -> 'washed'
 """)

We can then "parse" tokenised input sentences as follows:

In [ ]:
# define sentence and tokenize it
sent = 'I washed the dishes'
sent = nltk.word_tokenize(sent)
# use a parser to generate all possible syntax trees for the input sentence given our grammar
parser = nltk.ChartParser(grammar)
# print out all analyses
for tree in parser.parse(sent):
    nltk.Tree.fromstring(str(tree)).pretty_print()

And we can generate from the grammar as follows:

In [ ]:
for sentence in generate(grammar):
     print(' '.join(sentence))

Activity: Update the grammar so that it will parse "They washed the car". You can use the "|" symbol to allow multiple words or symbols on the right hand side of the rule, e.g. V -> 'washed' | 'threw'

Activity: Update the grammar so that it will parse "The boy and his dog enter the park". Note - it is permitted for the same terminal symbol to appear on the left and the right hand side of the same rule.

Activity: Generate from the grammar again. Why does it crash?

Activity: Update the grammar so that it will correctly parse the sentence "I washed the dishes on the counter". The intended interpretation is that the dishes were formerly on the counter and the washing took place in the sink. So the correct parse is as follows.

![washed](https://drive.google.com/uc?id=15zyDad-tHMG3pevk9kxOv3upmyOmLGbk)





In [ ]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N | Pronoun
    VP -> V NP 
    Det -> 'the' 
    Pronoun -> 'I'
    N -> 'dishes' 
    V -> 'washed'
 """)

In [ ]:
sent = 'I washed the dishes on the counter'
sent = nltk.word_tokenize(sent)
parser = nltk.ChartParser(grammar)
for tree in parser.parse(sent):
    nltk.Tree.fromstring(str(tree)).pretty_print()

Activity: now add rules to the same grammar to also give the correct analysis to the sentence "I washed my hair in the shower"

In [ ]:
sentences = ['I washed the dishes on the counter', 'I washed my hair in the shower']
parser = nltk.ChartParser(grammar)
for sent in sentences:
    for tree in parser.parse(nltk.word_tokenize(sent)):
        nltk.Tree.fromstring(str(tree)).pretty_print()

# Probabilistic Grammar
Because even very simple grammars can allow multiple, and sometimes a great many, analyses for simple sentences, particularly as the grammar gets big, it becomes necessary to find a way to prefer one parse over others. One way to accomplish this is with probabilistic grammars where a weight is given to each rule.

In [ ]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.25]
    NP -> NP PP [0.25]
    NP -> N PP [0.25]
    NP -> Pronoun [0.25]
    PP -> P NP [1.0]
    VP -> V NP [0.5]
    VP -> VP PP [0.5]
    Det -> 'the' [0.5]
    Det -> 'my' [0.5]
    Pronoun -> 'I' [1.0]
    N -> 'dishes'  [0.25]
    N -> 'sink' [0.25]
    N -> 'breakfast' [0.25]
    N -> 'pyjamas'[0.25]
    V -> 'washed' [0.5]
    V ->  'ate' [0.5]
    P -> 'in' [1.0]
 """)

In [ ]:
sentences = ['I ate my breakfast in my pyjamas', 'I washed the dishes in the sink']
parser = nltk.ViterbiParser(grammar)
import re
for sent in sentences:
    for tree in parser.parse_all(nltk.word_tokenize(sent)):
        tree = re.sub("\(p[^\)]+\)","",str(tree))
        nltk.Tree.fromstring(str(tree)).pretty_print()


Activity: Change the probabilities to assign the correct analysis for I washed the dishes in the sink

Getting the correct solution for both sentences at the same time requires an additional change to the form of the grammar. Any ideas what might work?

## Treebanks and grammar induction

Just writing these few small toy grammars has been quite involved. Writing full grammars that will have wide coverage is extremely difficult. We therefore learn them from corpora that have been annotated with syntax trees, known as treebanks.

Some treebanks are build into NLTK and we can load an example as follows:

In [ ]:
from nltk.corpus import treebank
nltk.download('treebank')

We can inspect an example tree as follows:

In [ ]:
t = treebank.parsed_sents('wsj_0001.mrg')[0]
nltk.Tree.fromstring(str(t)).pretty_print()

We can learn a grammar from treebank data as follows. 

First we have to make a slight change to the format of the trees:

In [ ]:
productions = []
for item in treebank.fileids():
  for tree in treebank.parsed_sents(item):
    # perform optional tree transformations, e.g.:
    tree.collapse_unary(collapsePOS = False)# Remove branches A-B-C into A-B+C
    tree.chomsky_normal_form(horzMarkov = 2)# Remove A->(B,C,D) into A->B,C+D->D
    productions += tree.productions()

And then we can "induce" a probabilistic grammar as follows.

In [ ]:
from nltk import induce_pcfg, grammar 
S = grammar.Nonterminal('S')
grammar_PCFG = induce_pcfg(S, productions)
print(grammar_PCFG)

In [ ]:
sentences = ['I drive in the city']
parser = nltk.ViterbiParser(grammar_PCFG)
import re
for sent in sentences:
    for tree in parser.parse_all(nltk.word_tokenize(sent)):
        tree = re.sub("\(p[^\)]+\)","",str(tree))
        nltk.Tree.fromstring(str(tree)).pretty_print()